In [5]:
PROJECT = "qwiklabs-gcp-ml-49b827b781ab"  # Replace with your PROJECT
BUCKET = "qwiklabs-gcp-ml-49b827b781ab"  # Replace with your BUCKET
REGION = "us-central1"            # Choose an available region for Cloud MLE
TFVERSION = "1.14"                # TF version for CMLE to use

In [6]:
import os
os.environ["BUCKET"] = BUCKET
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = TFVERSION

In [7]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
    gsutil mb -l ${REGION} gs://${BUCKET}
fi

In [8]:
!gsutil cp gs://cloud-training-demos/taxifare/small/*.csv .
!ls -l *.csv

Copying gs://cloud-training-demos/taxifare/small/taxi-test.csv...
Copying gs://cloud-training-demos/taxifare/small/taxi-train.csv...              
Copying gs://cloud-training-demos/taxifare/small/taxi-valid.csv...              
- [3 files][ 10.9 MiB/ 10.9 MiB]                                                
Operation completed over 3 objects/10.9 MiB.                                     
-rw-r--r-- 1 jupyter jupyter 1799474 Dec  6 11:40 taxi-test.csv
-rw-r--r-- 1 jupyter jupyter 7986353 Dec  6 11:40 taxi-train.csv
-rw-r--r-- 1 jupyter jupyter 1673742 Dec  6 11:40 taxi-valid.csv


# Create TensorFlow model using TensorFlow's Estimator API

In [9]:
import shutil
import numpy as np
import tensorflow as tf
print(tf.__version__)

1.15.0


In [10]:
CSV_COLUMNS = "weight_pounds,is_male,mother_age,plurality,gestation_weeks".split(',')
LABEL_COLUMN = "weight_pounds"

# Set default values for each CSV column
DEFAULTS = [[0.0], ["null"], [0.0], ["null"], [0.0]]
TRAIN_STEPS = 1000

In [11]:
def read_dataset(filename_pattern, mode, batch_size = 512):
    def _input_fn():
        def decode_csv(value_column):
            columns = tf.decode_csv(records = value_column, record_defaults = DEFAULTS)
            features = dict(zip(CSV_COLUMNS, columns))
            label = features.pop(LABEL_COLUMN)
            return features, label
    
        # Create list of files that match pattern
        file_list = tf.gfile.Glob(filename = filename_pattern)

        # Create dataset from file list
        dataset = (tf.data.TextLineDataset(filenames = file_list)  # Read text file
                     .map(map_func = decode_csv))  # Transform each elem by applying decode_csv fn

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None # indefinitely
            dataset = dataset.shuffle(buffer_size = 10 * batch_size)
        else:
            num_epochs = 1 # end-of-input after this

        dataset = dataset.repeat(count = num_epochs).batch(batch_size = batch_size)
        return dataset
    return _input_fn

# Create the feature columns

In [15]:
def get_categorical(name, values):
    return tf.feature_column.indicator_column(
        categorical_column = tf.feature_column.categorical_column_with_vocabulary_list(key = name, vocabulary_list = values))

def get_cols():
    # Define column types
    return [\
          get_categorical("is_male", ["True", "False", "Unknown"]),
          tf.feature_column.numeric_column(key = "mother_age"),
          get_categorical("plurality",
                      ["Single(1)", "Twins(2)", "Triplets(3)",
                       "Quadruplets(4)", "Quintuplets(5)","Multiple(2+)"]),
          tf.feature_column.numeric_column(key = "gestation_weeks")
    ]

# Create the Serving Input function

In [12]:
def serving_input_fn():
    feature_placeholders = {
        "is_male": tf.placeholder(dtype = tf.string, shape = [None]),
        "mother_age": tf.placeholder(dtype = tf.float32, shape = [None]),
        "plurality": tf.placeholder(dtype = tf.string, shape = [None]),
        "gestation_weeks": tf.placeholder(dtype = tf.float32, shape = [None])
    }
    
    features = {
        key: tf.expand_dims(input = tensor, axis = -1)
        for key, tensor in feature_placeholders.items()
    }
    
    return tf.estimator.export.ServingInputReceiver(features = features, receiver_tensors = feature_placeholders)

# Create the model and run training and evaluation

In [13]:
def train_and_evaluate(output_dir):
    EVAL_INTERVAL = 300
    
    run_config = tf.estimator.RunConfig(
        save_checkpoints_secs = EVAL_INTERVAL,
    keep_checkpoint_max = 3)
    
    estimator = tf.estimator.DNNRegressor(
        model_dir = output_dir,
        feature_columns = get_cols(),
        hidden_units = [64, 32],
        config = run_config)
    
    train_spec = tf.estimator.TrainSpec(
        input_fn = read_dataset("train.csv", mode = tf.estimator.ModeKeys.TRAIN),
        max_steps = TRAIN_STEPS)
    
    exporter = tf.estimator.LatestExporter(name = "exporter", serving_input_receiver_fn = serving_input_fn)
    
    eval_spec = tf.estimator.EvalSpec(
        input_fn = read_dataset("eval.csv", mode = tf.estimator.ModeKeys.EVAL),
        steps = None,
        start_delay_secs = 60, # start evaluating after N seconds
        throttle_secs = EVAL_INTERVAL,  # evaluate every N seconds
        exporters = exporter)
        
    tf.estimator.train_and_evaluate(estimator = estimator, train_spec = train_spec, eval_spec = eval_spec)

In [16]:
# Run the model
shutil.rmtree(path = "babyweight_trained_dnn", ignore_errors = True) # start fresh each time
train_and_evaluate("babyweight_trained_dnn")

INFO:tensorflow:Using config: {'_service': None, '_task_id': 0, '_task_type': 'worker', '_device_fn': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 300, '_is_chief': True, '_save_checkpoints_steps': None, '_train_distribute': None, '_num_worker_replicas': 1, '_evaluation_master': '', '_eval_distribute': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_experimental_distribute': None, '_num_ps_replicas': 0, '_global_id_in_cluster': 0, '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 3, '_session_creation_timeout_secs': 7200, '_protocol': None, '_model_dir': 'babyweight_trained_dnn', '_experimental_max_worker_delay_secs': None, '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff16aba3518>, '_log_step_count_steps': 100}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and e